# Advent of Code 2022 - Day 12

In [8]:
from collections import deque

In [9]:
# Read data
with open('input.txt', 'r') as f:
    data = f.read().strip().split('\n')

In [10]:
data[0:2]

['abccccccccccccccccaaccccccccccccccccccccaaaaaaaaaaaaacccccccccccccccccccccccccccccccccccccccccccccccccccccccaaaaaa',
 'abcccccccccccccaaaaaccccccccccccccccccccaaaaaaaaaaaaaccccccccccccccccccccccccccccccccccccccccccccccccccccccccaaaaa']

In [11]:
# Split each element in data into a list of characters
data = [[c for c in s] for s in data]

In [6]:
def find_pos(s):
    """
    Find row and column position of character 's' in the data list.
    """
    x, y = -999, -999
    if any(s in (match := nested_list) for nested_list in data):
        y = data.index(match)
        x = match.index(s)
    return [y, x]

In [7]:
S = find_pos('S')

In [8]:
E = find_pos('E')

In [9]:
S, E

([20, 0], [20, 91])

In [11]:
# Change values at start and end positions from (S,E) to their respective heights
data[S[0]][S[1]] = 1 # start position (lowest point possible)
data[E[0]][E[1]] = 26 # end position (highest point possible)

In [12]:
# Convert letters to numbers representing the heights
data = [[ord(char) - 96 if not isinstance(char, int) else char for char in s] for s in data]

In [14]:
R = len(data) # No. rows
C = len(data[0]) # No. columns
sr, sc = S[0], S[1] # start position
rq, cq = deque(), deque() # Row and column queues (could do together but this helps readability)

move_count = 0 # Number of moves before reaching end position
nodes_left_in_layer = 1 # Tracks how many nodes need to be dequeued before taking a step
nodes_in_next_layer = 0 # Tracks how many nodes were added in the breadth first search so we can update nodes_left_in_layer
                        # in the next iteration

reached_end = False # Tracks whether the highest point has been reached

visited = [[False for i in range(len(data[0]))] for j in range(len(data))] # Tracks which nodes have already been visited
                                                                           # (don't want to go back on ourselves)

dr = [-1, +1, 0, 0] # South and North directions
dc = [0, 0, +1, -1] # East and West directions

def explore_neighbours(r, c):
    """
    Checks each direction to see if that direction contains a node we can travel to.
    """
    global data, rq, cq, visited, nodes_in_next_layer
    for i in range(4):
        rr = r + dr[i] # Row of neighbour node being investigated
        cc = c + dc[i] # Column of neighbour node being investigated
        
        # Skip out of bounds nodes
        if rr < 0 or cc < 0: continue
        if rr >= R or cc >= C: continue
            
        # Skip nodes already visited
        if visited[rr][cc]: continue
            
        # Skip neighbour nodes with value not 1 greater than current value as we can't travel to those
        # We can however jump down to any node (so don't need < expression)
        if data[rr][cc] - data[r][c] > 1: continue
            
        # Can travel to this node so add it to queue
        rq.append(rr)
        cq.append(cc)
        visited[rr][cc] = True
        nodes_in_next_layer += 1 # Increment so we know how many nodes will be investigated next
        
def solve():
    """
    Manages overall queue and continues breadth first search until end is reached.
    """
    global data, rq, cq, sr, sc, visited, nodes_left_in_layer, nodes_in_next_layer, move_count, reached_end
    
    # Add start position to queue
    rq.append(sr)
    cq.append(sc)
    
    # Don't want to revisit start position
    visited[sr][sc] = True
    
    # Loop until queue is empty (i.e. no more neighbouring nodes to investigate)
    while rq:
        # Remove current node from front of queue
        r = rq.popleft()
        c = cq.popleft()
        
        # Check if end position is reached (technically checking whether point at same height as E is reached)
        if r == E[0] and c == E[1]:
            reached_end = True
            break
        
        # Check this nodes neighbours and add those we can travel to to the queue
        explore_neighbours(r, c)
        # Checked this node so decrement nodes left in layer by 1
        nodes_left_in_layer -= 1
        
        # If no more nodes left in this layer, move to next layer and increment move_count
        if nodes_left_in_layer == 0:
            nodes_left_in_layer = nodes_in_next_layer
            nodes_in_next_layer = 0
            move_count += 1
    
    # Once queue is empty, check we have reached the end position and return the number of moves
    if reached_end:
        return move_count
    else: return -1


## Part 1

In [15]:
sr, sc = S[0], S[1]
print(solve())

425


## Part 2

In [16]:
scores = [] # List to hold scores for all lowest elevation positions
# Create list of tuples showing index of each lowest elevation position
starts = [(i,j) for i in range(len(data)) for j in range(len(data[0])) if data[i][j]==1]

# Loop over starts
for (i,j) in starts:
    # Set new start position
    sr, sc = i, j
    # Run solver and append to list
    scores.append(solve())
    # Reset global variables that are edited by solve()
    move_count = 0
    reached_end = False
    nodes_left_in_layer = 1
    nodes_in_next_layer = 0
    visited = [[False for k in range(len(data[0]))] for l in range(len(data))]
    rq, cq = deque(), deque()

# Print minimum score that isn't equal to -1 (-1 means summit wasn't reached)
print(min([s for s in scores if s!=-1]))

418
